## **Advances in Data Mining**

Stephan van der Putten | (s1528459) | stvdputtenjur@gmail.com  
Theo Baart | s2370328 | s2370328@student.leidenuniv.nl

### **Assignment 1**
This assignment is concered with implementing formulas and models capable of predicting movie ratings for a set of users. Additionally, the accuracy of the various models are checked. 

#### **Naive Approaches**
This specific notebook handles the implementation of various naive approaches/formulas to the prediction problem.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### **Data Extraction**

The `extract_data` function is used to extract the data from the raw data file into a format useable within the formulas.

### **Rating Global**

The `rating_global` function predicts the user's ratings for a certain movie by taking the mean of all the ratings in the dataset. In order to do this the function uses the following parameters:
    * `User` - the user for which we want the rating
    * `Item` - the item (movie) for which we want the rating

### **Rating Item**

The `rating_item` function predicts the user's ratings for a certain movie by taking the mean of all the ratings in the dataset for that specific movie. In order to do this the function uses the following parameters:
    * `User` - the user for which we want the rating
    * `Item` - the item (movie) for which we want the rating

### **Rating User**

The `rating_user` function predicts the user's ratings for a certain movie by taking the mean of all the ratings in the dataset by the specific user. In order to do this the function uses the following parameters:
    * `User` - the user for which we want the rating
    * `Item` - the item (movie) for which we want the rating

### **Rating User-Item**

The `rating_user_item` function predicts the user's ratings for a certain movie by applying a linear regression to the outputs of the `rating_user` and `rating_item` functions. In order to do this the function uses the following parameters:
    * `User` - the user for which we want the rating
    * `Item` - the item (movie) for which we want the rating
    * `alpha` - the weight for the `rating_user` function
    * `beta` - the weight for the `rating_item` function
    * `gamma` - the offset/modifier